# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [8]:
#! pip install smart_open==1.9.0

### Import the libraries

In [9]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

In [9]:
from textacy.extract import keyterms

### Load the dataset

In [10]:
mytext = pd.read_json('newsgroups.json')
print(mytext.target_names.unique())
mytext.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


### Preprocess the data

### Email Removal

In [11]:
data = mytext.content.values.tolist()
#Email characters removal

In [12]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

In [13]:
pprint(data[:1])

["From: (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


### Newline Removal

In [14]:
data = [re.sub('\s+', ' ', sent) for sent in data]

### Single Quotes Removal

In [15]:
data = [re.sub("\'", "", sent) for sent in data]

In [16]:
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [18]:
from gensim.parsing.preprocessing import STOPWORDS, remove_stopwords, preprocess_string
my_stop_words = STOPWORDS.union(set(['from', 'subject', 're', 'edu', 'use']))

#### remove_stopwords( )

In [19]:
def remove_stops(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in my_stop_words] for doc in texts]

In [20]:
data_words_nostops = remove_stops(data_words)

In [21]:
print(data_words_nostops[:1])

[['wheres', 'thing', 'car', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'years', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'mail', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst']]


### Bigrams
- Use **gensim.models.Phrases**
- 100 as threshold

In [22]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

#### make_bigrams( )

In [23]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [24]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [25]:
data_words_bigrams = make_bigrams(data_words_nostops)

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [26]:
import spacy
nlp = spacy.load('en_core_web_sm')

#### lemmatizaton( )

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [29]:
print(data_lemmatized[:1])

[['thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'model', 'engine', 'spec', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'mail', 'bring', 'neighborhood', 'lerxst']]


### Create a Dictionary

In [30]:
id2word = corpora.Dictionary(data_lemmatized)

### Create Corpus

In [31]:
texts = data_lemmatized

### Filter low-frequency words

In [32]:
corpus = [id2word.doc2bow(text) for text in texts]

### Create Index 2 word dictionary

In [35]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('bumper', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('model', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('thing', 1),
  ('wonder', 1),
  ('year', 1)]]

In [36]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]]


### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [37]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### Print the Keyword in the 10 topics

In [38]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.088*"price" + 0.081*"sell" + 0.079*"sale" + 0.072*"player" + '
  '0.058*"cheap" + 0.053*"disk" + 0.027*"purchase" + 0.024*"market" + '
  '0.024*"motherboard" + 0.023*"manual"'),
 (1,
  '0.130*"program" + 0.123*"file" + 0.053*"entry" + 0.037*"server" + '
  '0.034*"distribution" + 0.024*"motif" + 0.022*"package" + 0.017*"mit" + '
  '0.017*"contain" + 0.015*"session"'),
 (2,
  '0.065*"drive" + 0.032*"card" + 0.026*"com" + 0.026*"software" + '
  '0.026*"driver" + 0.024*"bit" + 0.023*"chip" + 0.023*"do" + 0.022*"machine" '
  '+ 0.022*"run"'),
 (3,
  '0.024*"people" + 0.020*"reason" + 0.018*"believe" + 0.017*"evidence" + '
  '0.016*"mean" + 0.015*"state" + 0.014*"say" + 0.014*"right" + 0.014*"think" '
  '+ 0.013*"point"'),
 (4,
  '0.112*"israeli" + 0.053*"object" + 0.046*"font" + 0.043*"arab" + '
  '0.038*"wave" + 0.025*"own" + 0.020*"lift" + 0.017*"translate" + '
  '0.016*"australia" + 0.014*"postscript"'),
 (5,
  '0.031*"car" + 0.027*"high" + 0.024*"power" + 0.018*"cost" + 0.017*

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [39]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -14.032929239418733


### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [40]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5169368069811037


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [51]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.287802  0.043139       1        1  16.904398
10     0.306535 -0.094037       2        1  16.636622
3      0.301786 -0.128210       3        1  15.531395
15     0.219634  0.099893       4        1  12.183473
5      0.110624  0.191172       5        1   6.105882
19     0.132333 -0.165480       6        1   5.797531
2      0.058244  0.272289       7        1   4.894525
9     -0.068680  0.003138       8        1   4.701238
11     0.049115 -0.180310       9        1   3.253761
13    -0.042037  0.083396      10        1   2.654787
7     -0.032260  0.030495      11        1   2.465809
1     -0.090305  0.105139      12        1   1.825993
18    -0.087228 -0.064574      13        1   1.615171
0     -0.129430  0.003084      14        1   1.137717
17    -0.121050 -0.031655      15        1   1.064726
6     -0.157564 -0.038545      16        1   1.008761
12    -0.137321  0.003407      17        1   0.989302
8     -0.184507 -0.046326      18        1   0.650762
4     -0.204149 -0.041354      19        1   0.353337
16    -0.211544 -0.044663      20        1   0.224810, topic_info=               Term          Freq         Total Category  logprob  loglift
4339             ax  42804.000000  42804.000000  Default  30.0000  30.0000
18             line  18261.000000  18261.000000  Default  29.0000  29.0000
194           write  14162.000000  14162.000000  Default  28.0000  28.0000
133    organization  11222.000000  11222.000000  Default  27.0000  27.0000
40          article   9950.000000   9950.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
4999         hockey      0.033916      1.389207  Topic20 -11.2745   2.3851
6066   clipper_chip      0.033915      1.389125  Topic20 -11.2746   2.3851
7011        encrypt      0.033916      1.389195  Topic20 -11.2745   2.3851
16031             c      0.033915      1.389159  Topic20 -11.2746   2.3851
4289           king      0.033915      1.389128  Topic20 -11.2746   2.3851

[859 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1333     17  0.997867        accept
79        7  0.771048        access
79       13  0.227479        access
3100     17  0.983461  access_digex
1726      3  0.955098        action
...     ...       ...           ...
35        2  0.961329          year
35        4  0.027930          year
35        9  0.010568          year
588       6  0.997747         young
2171     16  0.994297          zone

[1083 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 11, 4, 16, 6, 20, 3, 10, 12, 14, 8, 2, 19, 1, 18, 7, 13, 9, 5, 17])